In [0]:
DECLARE catalogo STRING;
DECLARE paciente STRING;
SET var catalogo = concat('workshop_databricks_', :iniciais);

USE CATALOG IDENTIFIER(catalogo);
USE SCHEMA agents;
--###################################################################################################
-- PREPARANDO A TABELA PACIENTES USANDO COPY INTO COM FORCE=TRUE, PARA VIABILIZAR O REPROCESSAMENTO #
--###################################################################################################
TRUNCATE TABLE pacientes;
COPY INTO pacientes
FROM '/Volumes/<NOME_DO_SEU_CATALOGO>/agents/vol_agent/pacientes.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
COPY_OPTIONS (
 'force' = 'true'
);
--############################################################
-- PREPARANDO DADOS DA TABELA MEDICOS, UTILIZANDO READ_FILES #
--############################################################
TRUNCATE TABLE medicos;
INSERT INTO medicos
SELECT nome_medico,endereco,cidade,estado,pais,cep,latitude,longitude, especialidade, from_json(planos_aceitos,'ARRAY<STRING>') planos_aceitos, bairro FROM read_files('/Volumes/<NOME_DO_SEU_CATALOGO>/agents/vol_agent/medicos.csv',
  format => 'csv',
  quote => '"',
  escape => '"',
  header => true
);

--######################################################
-- VERIFICANDO SE AMBAS TABELAS POSSUEM 1000 REGISTROS #
--######################################################
SELECT 'A tabela tabela médicos tem: '|| string(count(1))|| ' registros' resultado from medicos
UNION ALL
SELECT 'A tabela pacientes tem: '|| string(count(1) || ' registros') from pacientes